# Análisis Financiero del Russell 1000 usando YahooQuery

## Etapa 1: Obtener listado del Russell 1000

In [1]:
import pandas as pd
import requests
import os
# URL de la página
url = 'https://en.wikipedia.org/wiki/Russell_1000_Index'

# Encabezado para evitar el error 403
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'
}

# Hacer la solicitud
response = requests.get(url, headers=headers)

# Leer las tablas HTML desde el contenido descargado
tables = pd.read_html(response.text)

# Inspeccionar la tabla 3 (asegúrate de que contenga los tickers)
df_r1000 = tables[3]  # O cambia a otro índice si no es la tabla correcta

# Mostrar las columnas disponibles
print(df_r1000.columns)

# Extraer los tickers si existe la columna 'Symbol'
if 'Symbol' in df_r1000.columns:
    tickers = df_r1000['Symbol'].tolist()
    tickers=tickers[:100]
else:
    print("❌ La columna 'Symbol' no está en esta tabla.")


Index(['Company', 'Symbol', 'GICS Sector', 'GICS Sub-Industry'], dtype='object')


C:\Users\juans\AppData\Local\Temp\ipykernel_20360\1279450636.py:16: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)


## Etapa 2: Descargar EPS básico (últimos 4 años fiscales)

In [2]:
from yahooquery import Ticker

# Obtener datos financieros para todos los tickers de una vez
t = Ticker(tickers)
types=['BasicEPS','CashCashEquivalentsAndShortTermInvestments', 'EBITDA', 'TotalRevenue', 'InterestExpense', 'TaxProvision', 'ChangeInWorkingCapital','LongTermDebtAndCapitalLeaseObligation']
eps_data = t.get_financial_data(types, trailing=False)


## Etapa 3: Filtrar empresas con 4 años de EPS positivo

In [3]:
# Filtrar empresas con valores negativos en variables críticas
eps_data = eps_data[
    (eps_data['EBITDA'] > 0) &
    (eps_data['TotalRevenue'] > 0)
]
# Reemplazar InterestExpense nulos por 0 (caso típico)
eps_data['InterestExpense'] = eps_data['InterestExpense'].fillna(0)
eps_data['CashCashEquivalentsAndShortTermInvestments'] = eps_data['CashCashEquivalentsAndShortTermInvestments'].fillna(0)
eps_data['TaxProvision'] = eps_data['TaxProvision'].fillna(0)
eps_data['LongTermDebtAndCapitalLeaseObligation'] = eps_data['LongTermDebtAndCapitalLeaseObligation'].fillna(0)

#Margin Ebitda
eps_data['EBITDA_Margin'] = eps_data['EBITDA'] / eps_data['TotalRevenue']
#Límite 1
Den = eps_data['EBITDA'] - eps_data['TaxProvision'] - eps_data['ChangeInWorkingCapital']
eps_data['Límite1'] = eps_data['InterestExpense'] / Den

#Límite 2
num = eps_data['LongTermDebtAndCapitalLeaseObligation'] - eps_data['CashCashEquivalentsAndShortTermInvestments']
Den2 = eps_data['EBITDA']
eps_data['Límite2'] = num / Den

# Mantener solo tickers con al menos 4 registros de EPS
ticker_counts = eps_data.index.get_level_values(0).value_counts()
tickers_validos = ticker_counts[ticker_counts >= 4].index.tolist()

# Filtrar dataset para dejar solo esas empresas
eps_filtrado = eps_data.loc[tickers_validos]


## Etapa 4: Reorganizar el EPS en un DataFrame limpio

In [4]:
# Resetear índice y limpiar columnas
eps_filtrado = eps_filtrado.reset_index()
eps_filtrado = eps_filtrado[['symbol', 'asOfDate', 'BasicEPS', 'EBITDA_Margin', 'Límite1','Límite2']]
eps_filtrado['asOfDate'] = pd.to_datetime(eps_filtrado['asOfDate'])

# Agregar filas con +3 días para cubrir posibles feriados/no cotización
import datetime

rows_extra = []
for idx, row in eps_filtrado.iterrows():
    new_date = row['asOfDate'] + datetime.timedelta(days=3)
    rows_extra.append({'symbol': row['symbol'], 'asOfDate': new_date, 'BasicEPS': row['BasicEPS'],'EBITDA_Margin': row['EBITDA_Margin'],'Límite1': row['Límite1'],'Límite2': row['Límite2']})

eps_fiscal = pd.concat([eps_filtrado, pd.DataFrame(rows_extra)], ignore_index=True)


## Etapa 5: Descargar precios históricos (últimos 5 años)

In [5]:

from datetime import datetime, timedelta
import os


START_DATE = "2020-05-30"          # Fecha inicial fija para descargar históricos
WEEKS_BACK = 2                     # Cuántas semanas antes de hoy descargar
PARQUET_FILE = "prices.parquet"    # Archivo local donde se guardan los datos


# CALCULAR FECHA FINAL (end_date)
# Dos semanas antes de la fecha actual
# .normalize() asegura que no tenga hora, para evitar problemas al comparar días

today = pd.Timestamp.today().normalize()
end_date = today - timedelta(days=WEEKS_BACK * 7)

# Ajustar end_date al último día hábil (lunes-viernes)
while end_date.weekday() > 4:
    end_date -= timedelta(days=1)

# SI EL ARCHIVO YA EXISTE, CARGARLO Y VERIFICAR SI NECESITA ACTUALIZACIÓN

if os.path.exists(PARQUET_FILE):

    # Cargar datos existentes
    prices = pd.read_parquet(PARQUET_FILE)

    # Última fecha disponible en el archivo local (sin hora)
    last_date = prices["asOfDate"].max().normalize()

    # Si el archivo ya está actualizado, no se descarga nada
    if last_date >= end_date:
        print(f"Datos actualizados hasta {last_date.date()}, no se descarga nada.")

    else:
        
        print(f"Actualizando datos desde {last_date.date()} hasta {end_date.date()}...")

        # Lista de tickers válidos ya almacenados
        tickers_validos = prices["symbol"].unique().tolist()

        # Crear objeto Ticker
        t = Ticker(tickers_validos)

        # Descargar solo la parte faltante
        new_data = t.history(
            start=last_date + timedelta(days=1),  # día siguiente a los datos existentes
            end=end_date,
            interval="1d"
        )

        # Limpiar columnas y formatear
        new_data = (
            new_data
            .drop(columns=['open', 'high', 'low', 'volume', 'adjclose', 'dividends', 'splits'])
            .reset_index()[['symbol', 'date', 'close']]
            .rename(columns={'date': 'asOfDate'})
        )

        # Normalizar fecha (sin hora)
        new_data['asOfDate'] = pd.to_datetime(new_data['asOfDate']).dt.normalize()

        # Combinar histórico viejo + nuevo, eliminando duplicados
        prices = pd.concat([prices, new_data], ignore_index=True).drop_duplicates()

        # Guardar archivo actualizado
        prices.to_parquet(PARQUET_FILE, index=False)

else:
  
    # PRIMERA DESCARGA (si no existe el archivo)

    print("Descargando datos históricos por primera vez...")

    # t_valid ya debería existir antes de esta etapa con los tickers válidos finales
    t = Ticker(tickers_validos)

    # Descargar histórico completo una sola vez
    prices = t.history(start=START_DATE, end=end_date, interval="1d")

    # Limpiar columnas
    prices = (
        prices
        .drop(columns=['open', 'high', 'low', 'volume', 'adjclose', 'dividends', 'splits'])
        .reset_index()[['symbol', 'date', 'close']]
        .rename(columns={'date': 'asOfDate'})
    )

    # Normalizar fecha (sin hora)
    prices['asOfDate'] = pd.to_datetime(prices['asOfDate']).dt.normalize()

    # Guardar archivo
    prices.to_parquet(PARQUET_FILE, index=False)

print("Proceso completado.")



Datos actualizados hasta 2025-11-14, no se descarga nada.
Proceso completado.


## Etapa 6: Unir EPS con precios para calcular PER

In [6]:
# Merge con símbolo y fecha
df_merge = pd.merge(prices, eps_fiscal, on=['symbol', 'asOfDate'])

# Filtrar empresas con 4 años válidos
valid_counts = df_merge['symbol'].value_counts()
df_merge = df_merge[df_merge['symbol'].isin(valid_counts[valid_counts >= 4].index)]

# Calcular PER
df_merge['PER'] = df_merge['close'] / df_merge['BasicEPS']


## Etapa 7: Calcular promedio EPS, PER y precio objetivo

In [21]:
Resumen = df_merge.sort_values(['symbol', 'asOfDate'])

promedios = (
    Resumen.groupby('symbol')
    .agg(PER_promedio=('PER', 'mean'),
         EPS_promedio=('BasicEPS', 'mean'))
    .reset_index()
)

# Calcular precio objetivo con PER promedio y EPS promedio
promedios['Precio_Objetivo'] = promedios['EPS_promedio'] * promedios['PER_promedio']
promedios['Precio_Objetivo_85'] = promedios['Precio_Objetivo'] * 0.85

# Filtrar empresas con PER promedio < 40
promedios_filtrados = promedios[promedios['PER_promedio'] < 40]

# Obtener lista de símbolos válidos
tickers_filtrados = promedios_filtrados['symbol'].tolist()

## Etapa 8: Obtener precios actuales

In [22]:
from datetime import date
t_validf = Ticker(tickers_filtrados)
precios_actuales = t_validf.history(start='2025-09-24', end='2025-09-25', interval='1d')
precios_actuales = precios_actuales.reset_index()[['symbol', 'close']]
precios_actuales = precios_actuales.groupby('symbol').last().reset_index()
precios_actuales.rename(columns={'close': 'Precio_Actual'}, inplace=True)


## Etapa 9: Resultado final y exportación

In [23]:
df_final = pd.merge(promedios_filtrados, precios_actuales, on='symbol', how='inner')

# Reordenar columnas y guardar
df_final = df_final[['symbol', 'Precio_Actual', 'Precio_Objetivo_85', 'PER_promedio', 'EPS_promedio']]
df_final.columns = ['Empresa', 'Precio Actual', 'Precio Objetivo', 'PER Promedio', 'EPS Promedio']

# Exportar
df_final.to_excel('Russell_1000_Valoraciones.xlsx', index=False)
print("Archivo guardado como 'Russell_1000_Valoraciones.xlsx'")


Archivo guardado como 'Russell_1000_Valoraciones.xlsx'


# Etapa 10 – Evolución anual del Margen EBITDA por empresa

In [24]:


# Filtrar eps_fiscal con los tickers que tienen PER promedio < 40
eps_filtrado_final = eps_fiscal[eps_fiscal['symbol'].isin(tickers_filtrados)].copy()

# Agregar columna de año
eps_filtrado_final['Año'] = eps_filtrado_final['asOfDate'].dt.year

# Calcular promedios anuales
margen_anual = eps_filtrado_final.groupby(['symbol', 'Año'])[['EBITDA_Margin', 'Límite1']].mean().reset_index()
margen_anual.rename(columns={
    'symbol': 'Empresa',
    'EBITDA_Margin': 'Margen EBITDA',
    'Límite1': 'Límite1'
}, inplace=True)

# Exportar a nueva hoja en el Excel existente
from openpyxl import load_workbook
with pd.ExcelWriter('Russell_1000_Valoraciones.xlsx', mode='a', engine='openpyxl') as writer:
    margen_anual.to_excel(writer, sheet_name='Margen EBITDA por Año', index=False)

print("Guardado 'Margen EBITDA por Año' con empresas que tienen PER Promedio < 40.")


Guardado 'Margen EBITDA por Año' con empresas que tienen PER Promedio < 40.
